In [1]:
from skimage import io,transform
import glob
import os
import tensorflow as tf
import numpy as np
import time
# path='flower_photos'
#the min image size
w=64
h=64

# read the data A after propressed

In [2]:
from skimage import io,transform
path='./dataresize128A'
files=os.listdir(path)
f_train=open('./train.txt')
content=f_train.readlines()
x_train1=[]
y_train1=[]
y_train2=[]
x_train2=[]
for i in content:
    [train_path,label]=i.split()
    im=io.imread(path+'/'+train_path+'.png')
    im=transform.resize(im,[64,64,1])
    x_train1.append(im)
    x_train2.append(im)
    y_train1.append(0)
    y_train2.append(label)

D:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [3]:
print(np.shape(x_train2))
print(np.shape(y_train1))
print(np.shape(y_train2))

(410, 64, 64, 1)
(410,)
(410,)


# read the data B after propressed

In [4]:
path='./dataresize64B'
files=os.listdir(path)
f_train=open('./label_B.txt')
content=f_train.readlines()
x_test=[]
y_test=[]
for i in content:
    [train_path,label]=i.split()
    im=io.imread(path+'/'+train_path+'.png')
    im=transform.resize(im,[64,64,1])
    x_train1.append(im)
    x_test.append(im)
    y_test.append(label)
    y_train1.append(1)

D:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
print(np.shape(x_test))
print(np.shape(y_test))
print(np.shape(x_train1))
print(np.shape(y_train1))
print(np.shape(x_train2))
print(np.shape(y_train2))

(322, 64, 64, 1)
(322,)
(732, 64, 64, 1)
(732,)
(410, 64, 64, 1)
(410,)


In [6]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,GlobalAveragePooling2D
import keras
from numpy import array

Using TensorFlow backend.


In [7]:
x_train1=array(x_train1)
x_train1=x_train1.astype('float32')
x_test=array(x_test)
x_test=x_test.astype('float32')
x_train2=array(x_train2)
x_train2=x_train2.astype('float32')

In [8]:
y_train1=keras.utils.to_categorical(y_train1,2)
y_train2=keras.utils.to_categorical(y_train2,2)
y_test=keras.utils.to_categorical(y_test,2)

In [9]:
input_shape=[64,64,1]

# model


In [92]:
# model=Sequential()
# # model.add(GlobalAveragePooling2D(input_shape=input_shape))
# model.add(Conv2D(32, 3, padding='same',input_shape=input_shape))
# model.add(Activation('relu'))

# model.add(MaxPooling2D(pool_size=3))

In [93]:
# model.add(Conv2D(64, 3, padding='same',strides=2))
# model.add(Activation('relu'))
# model.add(Conv2D(64, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(64, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=2))

In [94]:
# model.add(Conv2D(128, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(128, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(128, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=2))

In [95]:
# model.add(Conv2D(128, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(128, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(128, 3, padding='same',strides=2))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=2))


In [96]:
# model.add(Conv2D(256, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(256, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(256, 3, padding='same'))
# model.add(Activation('relu'))
# model.add(GlobalAveragePooling2D())

# use the first 14 layers of vgg16 to pretrain image features

In [10]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
model= VGG16(weights='imagenet', include_top=False)
# test=model.predict(x_train3)
test=model.get_layer('block5_conv2').output

In [11]:
# model.summary()
model_vgg14=Model(inputs=model.input,outputs=test)

In [12]:
model_vgg14.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [13]:
np.shape(x_train2)

(410, 64, 64, 1)

In [50]:
x_train3=np.concatenate([x_train2,x_train2,x_train2],3)
test_train1=model_vgg14.predict(x_train3)
x_train5=np.concatenate([x_train1,x_train1,x_train1],3)
test_train2=model_vgg14.predict(x_train5)

In [15]:
x_train4=np.concatenate([x_test,x_test,x_test],3)
test_test=model_vgg14.predict(x_train4)

In [16]:
print(np.shape(test))

(410, 4, 4, 512)


# add two layers to fine tune on our own dataset

In [17]:
model_fe=Sequential()
input_shape=[4,4,512]

model_fe.add(Conv2D(512, 2, padding='same',input_shape=input_shape))
model_fe.add(Activation('relu'))

model_fe.add(MaxPooling2D(pool_size=2))

model_fe.add(Conv2D(512, 2, padding='same',input_shape=input_shape))
model_fe.add(Activation('relu'))

model_fe.add(MaxPooling2D(pool_size=2))

# model_22.add(Flatten())
# model_22.add(Dense(512,input_shape=input_shape)) 
# model_22.add(Activation('relu')) 

# model_22.add(Dense(2))  
# model_22.add(Activation('softmax'))

In [18]:
model_fe.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['binary_accuracy'])

In [81]:
# pre=model_22.fit(test,y_train2,epochs=200,shuffle=True)

# build domain model and predict model

In [33]:
test1=model_fe.output
test1=Flatten()(test1)
test1=Dense(512,activation='relu')(test1)
test1=Dense(2,activation='softmax')(test1)
model_domain=Model(inputs=model_fe.input,outputs=test1)

In [21]:
test2=model_fe.output
test2=Flatten()(test2)
test2=Dense(512,activation='relu')(test2)
test2=Dense(2,activation='softmax')(test2)
model_B=Model(inputs=model_fe.input,outputs=test2)

In [44]:
from keras import optimizers
sgd =-optimizers.SGD(lr=0.01, decay=-1e-6, momentum=0.9, nesterov=True)

AttributeError: 'SGD' object has no attribute 'graident'

In [46]:
model_domain.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [92]:
model_B.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['binary_accuracy'])

In [97]:
model_domain.fit(test_train2,y_train1,epochs=100,shuffle=True)

# train the predict model on data A

In [98]:
model_B.fit(test_train1,y_train2,epochs=100,shuffle=True)



Epoch 1/100
410/410 [==============================] - 3s 7ms/step - loss: 0.5900 - binary_accuracy: 0.7805
Epoch 2/100
410/410 [==============================] - 3s 7ms/step - loss: 0.3489 - binary_accuracy: 0.8488
Epoch 3/100
410/410 [==============================] - 3s 7ms/step - loss: 0.3236 - binary_accuracy: 0.8707
Epoch 4/100
410/410 [==============================] - 3s 7ms/step - loss: 0.2483 - binary_accuracy: 0.8780
Epoch 5/100
410/410 [==============================] - 3s 7ms/step - loss: 0.3549 - binary_accuracy: 0.8537
Epoch 6/100
410/410 [==============================] - 3s 8ms/step - loss: 0.2881 - binary_accuracy: 0.9000
Epoch 7/100
410/410 [==============================] - 3s 7ms/step - loss: 0.2691 - binary_accuracy: 0.8805
Epoch 8/100
410/410 [==============================] - 3s 6ms/step - loss: 0.5153 - binary_accuracy: 0.8024
Epoch 9/100
410/410 [==============================] - 3s 6ms/step - loss: 0.4064 - binary_accuracy: 0.8439
Epoch 10/100
410/410 [======

410/410 [==============================] - 3s 7ms/step - loss: 0.4775 - binary_accuracy: 0.8220
Epoch 76/100
410/410 [==============================] - 2s 6ms/step - loss: 0.3371 - binary_accuracy: 0.8659
Epoch 77/100
410/410 [==============================] - 2s 6ms/step - loss: 0.3099 - binary_accuracy: 0.8829
Epoch 78/100
410/410 [==============================] - 3s 6ms/step - loss: 0.3660 - binary_accuracy: 0.8634
Epoch 79/100
410/410 [==============================] - 3s 7ms/step - loss: 0.3284 - binary_accuracy: 0.8683
Epoch 80/100
410/410 [==============================] - 3s 6ms/step - loss: 0.3707 - binary_accuracy: 0.8366
Epoch 81/100
410/410 [==============================] - 3s 6ms/step - loss: 0.3028 - binary_accuracy: 0.8537
Epoch 82/100
410/410 [==============================] - 2s 6ms/step - loss: 0.3790 - binary_accuracy: 0.8341
Epoch 83/100
410/410 [==============================] - 3s 7ms/step - loss: 0.2687 - binary_accuracy: 0.8780
Epoch 84/100
410/410 [==========

# pre

In [94]:

print(np.shape(test_test))
print(np.shape(y_test))
score=model_B.evaluate(test_test,y_test)

(322, 4, 4, 512)
(322, 2)
322/322 [==============================] - 1s 2ms/step


In [95]:
score

[1.5540010581105392, 0.5]

In [105]:
model_domain.fit(test,y_train1,epochs=5,shuffle=True)

Epoch 1/5
732/732 [==============================] - 5s 6ms/step - loss: 0.0019 - acc: 1.0000
Epoch 2/5
732/732 [==============================] - 4s 6ms/step - loss: 9.6484e-04 - acc: 1.0000
Epoch 3/5
732/732 [==============================] - 4s 6ms/step - loss: 5.7375e-04 - acc: 1.0000
Epoch 4/5
732/732 [==============================] - 4s 6ms/step - loss: 3.8717e-04 - acc: 1.0000
Epoch 5/5
732/732 [==============================] - 4s 6ms/step - loss: 3.1165e-04 - acc: 1.0000


In [ ]:
test=model_22.get_layer('').output

model_3=Sequential()
input_shape=[4,4,512]

model_3.add(Conv2D(512, 2, padding='same',input_shape=input_shape))
model_3.add(Activation('relu'))

model_3.add(MaxPooling2D(pool_size=2))

model_22.add(Conv2D(512, 2, padding='same',input_shape=input_shape))
model_22.add(Activation('relu'))

model_22.add(MaxPooling2D(pool_size=2))

# model_22.add(Flatten())
# model_22.add(Dense(512,input_shape=input_shape)) 
# model_22.add(Activation('relu')) 

# model_22.add(Dense(2))  
# model_22.add(Activation('softmax'))

In [217]:
# input1=keras.layers.Input(shape=(64,64,1))
# x1 = keras.layers.Conv2D(256, 3, padding='same',activation='relu')(input1)
# x1 = keras.layers.Conv2D(128, 3, padding='same',activation='relu')(x1)
# x1 = keras.layers.Conv2D(128, 3, padding='same',activation='relu')(x1)
# x1 = keras.layers.Conv2D(64, 3, padding='same',activation='relu')(x1)
# x1=reverse_gradient(x1, 1)

In [218]:
# x1=keras.layers.Dense(512,activation='relu')(x1)  

In [ ]:
x=model.output
# x=Flatten()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(2, activation='softmax')(x)
model2=Model(inputs=model.input,outputs=predictions)

In [219]:
# x1=keras.layers.Dense(1,activation='softmax')(x1) 

In [62]:
x=model.output
# x=Flatten()(x)
x=Dense(512,activation='relu')(x)
predictions=Dense(2, activation='softmax')(x)
model1=Model(inputs=model.input,outputs=predictions)

In [98]:
x=model.output
# x=Flatten()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(2, activation='softmax')(x)
model2=Model(inputs=model.input,outputs=predictions)

In [81]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40_input (InputLayer) (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 256, 256, 32)      320       
_________________________________________________________________
activation_40 (Activation)   (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 43, 43, 64)        18496     
_________________________________________________________________
activation_41 (Activation)   (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 43, 43, 64)        36928     
__________

In [222]:
# model.add(Flatten())  
# model.add(Dense(512))  
# model.add(Activation('relu'))  
# # model.add(Dropout(0.5))

# model.add(Dense(2))  
# model.add(Activation('softmax'))


In [39]:
model1.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

NameError: name 'model1' is not defined

In [73]:
domain=model1.fit(x_train1,y_train1,epochs=1,shuffle=True)

Epoch 1/1
732/732 [==============================] - 56s 77ms/step - loss: 0.6533 - acc: 0.5642


In [99]:
model2.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [100]:
print(np.shape(x_train2))

(710, 256, 256, 1)


In [101]:
pre=model2.fit(x_train2,y_train2,epochs=5,shuffle=True)

Epoch 1/5
710/710 [==============================] - 101s 143ms/step - loss: 0.6918 - acc: 0.5634
Epoch 2/5
710/710 [==============================] - 104s 146ms/step - loss: 0.6886 - acc: 0.5634
Epoch 3/5
160/710 [=====>........................] - ETA: 2:33 - loss: 0.6804 - acc: 0.6250

KeyboardInterrupt: 

In [31]:
from keras.models import Model
model_21=Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_2').output)

In [32]:
features=model_21.predict(x_train2)

In [33]:

print(np.shape(features))

(410, 16, 16, 128)


In [36]:
input_shape=[16,16,128]
model_22=Sequential()

model_22.add(Conv2D(128, 3, padding='same', input_shape=input_shape))
model_22.add(Activation('relu'))
model_22.add(MaxPooling2D(pool_size=2))

model_22.add(Conv2D(64, 3, padding='same'))
model_22.add(Activation('relu'))
model_22.add(MaxPooling2D(pool_size=2))

model_22.add(Flatten(input_shape=input_shape))
model_22.add(Dense(512)) 
model_22.add(Activation('relu'))
model.add(Dropout(0.5))

In [37]:
model_22.add(Dense(512)) 
model_22.add(Activation('relu')) 
model.add(Dropout(0.5))
model_22.add(Dense(2))  
model_22.add(Activation('softmax'))

In [38]:
model_22.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
activation_10 (Activation)   (None, 16, 16, 128)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 64)          73792     
_________________________________________________________________
activation_11 (Activation)   (None, 8, 8, 64)          0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
__________

In [39]:
model_22.compile(loss='hinge',  
              optimizer='sgd',
              metrics=['accuracy'])

In [40]:
pre=model_22.fit(features,y_train2,epochs=10,shuffle=True,batch_size=10)

Epoch 1/10
410/410 [==============================] - 3s 8ms/step - loss: 0.6974 - acc: 0.7561
Epoch 2/10
410/410 [==============================] - 3s 8ms/step - loss: 0.6438 - acc: 0.7561
Epoch 3/10
410/410 [==============================] - 3s 8ms/step - loss: 0.6297 - acc: 0.7561
Epoch 4/10
410/410 [==============================] - 3s 8ms/step - loss: 0.6260 - acc: 0.7561
Epoch 5/10
410/410 [==============================] - 3s 8ms/step - loss: 0.6245 - acc: 0.7561
Epoch 6/10
410/410 [==============================] - 3s 8ms/step - loss: 0.6238 - acc: 0.7561
Epoch 7/10
410/410 [==============================] - 3s 8ms/step - loss: 0.6234 - acc: 0.7561
Epoch 8/10
410/410 [==============================] - 3s 8ms/step - loss: 0.6231 - acc: 0.7561
Epoch 9/10
410/410 [==============================] - 3s 8ms/step - loss: 0.6229 - acc: 0.7561
Epoch 10/10
410/410 [==============================] - 4s 9ms/step - loss: 0.6227 - acc: 0.7561


In [123]:
test_features=model_21.predict(x_test)

In [124]:
score=model_22.evaluate(test_features,y_test)
print(score)

322/322 [==============================] - 0s 1ms/step
[5.7250854190976215, 0.6428571428571429]


In [23]:
-